In [1]:
#import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.colors as colors
from matplotlib import rc
plt.rcParams["font.serif"]       = 'Times New Roman'
plt.rcParams["xtick.major.size"] = 10
plt.rcParams["ytick.major.size"] = 10
plt.rcParams["xtick.minor.size"] = 5
plt.rcParams["ytick.minor.size"] = 5
plt.rcParams["xtick.labelsize"]  = 35
plt.rcParams["ytick.labelsize"]  = 35

plt.rcParams["legend.title_fontsize"]=35
import sys
import string

In [2]:
flag_define_ESurfInt=1 #1 if you want to define surfaces to obtain efield surface integral else 0
flag_define_EVolumeInt=1 #1 if you want to define volumes to obtain eField volume integral else 0
flag_define_BSurfInt=1 #1 if you want to define surfaces to obtain bfield surface integral else 0
flag_define_BVolumeInt=1 #1 if you want to define volumes to obtain bField volume integral else 0
fileName='fieldReductionInput_1x.dat'
numYLocs_ESurfInt = 10
numYLocs_BSurfInt = numYLocs_ESurfInt
numEVolumes=1
numBVolumes=1

In [3]:
sub_thick=5e-4
strip_width=0.97518*sub_thick
strip_width_inv=round(1./strip_width,3)
mu_0=1.25663706212e-06
mu_0_inv = round(1./mu_0,4)
print(strip_width_inv)
print(mu_0_inv)

2050.903
795774.715



# Define EField contour integral around a conductor using RawEFieldReduction

In [4]:
numSurfsPerYLoc_ESurfInt=1
numSurfs_ESurfInt = numYLocs_ESurfInt*numSurfsPerYLoc_ESurfInt
eSurfName=np.empty(numSurfs_ESurfInt, dtype=object)
reduced_diags_name_str = 'warpx.reduced_diags_names = ' 
reduced_diags_names_efield_surface_str = ''
for l in range(0, numYLocs_ESurfInt):
    for s in range(0, numSurfsPerYLoc_ESurfInt):
        eSurfName[l*numSurfsPerYLoc_ESurfInt+s] = 'ESurf_L'+ str(l+1)+'S'+str(s+1)
        reduced_diags_names_efield_surface_str += eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + ' '
print(reduced_diags_name_str + reduced_diags_names_efield_surface_str)


#Choose distance
#numUniformYLocs_BSurfInt=numYLocs_BSurfInt-10
#distance_of_first_10={'dy', '3*dy', '6*dy', '9*dy', '12*dy', '15*dy', '18*dy', '21*dy', '24*dy', '27*dy', '30*dy'}

distance_uniform_ESurfInt=np.empty(numYLocs_ESurfInt, dtype=object)
distance_uniform_ESurfInt[0]='dy'
distance_uniform_ESurfInt[numYLocs_ESurfInt-1]='Ly-2*dy'
remainingLocs_ESurfInt=numYLocs_ESurfInt-2
percent_step_ESurfInt = 100./(remainingLocs_ESurfInt+1)

for l in range(1, numYLocs_ESurfInt-1):
    distance_uniform_ESurfInt[l]= str(percent_step_ESurfInt*(l)/100) + '*Ly'
    
print('locations: ', distance_uniform_ESurfInt)

warpx.reduced_diags_names = ESurf_L1S1 ESurf_L2S1 ESurf_L3S1 ESurf_L4S1 ESurf_L5S1 ESurf_L6S1 ESurf_L7S1 ESurf_L8S1 ESurf_L9S1 ESurf_L10S1 
locations:  ['dy' '0.1111111111111111*Ly' '0.2222222222222222*Ly'
 '0.33333333333333326*Ly' '0.4444444444444444*Ly' '0.5555555555555556*Ly'
 '0.6666666666666665*Ly' '0.7777777777777777*Ly' '0.8888888888888888*Ly'
 'Ly-2*dy']


In [5]:
eSurf_type_str = ".type = RawEFieldReduction"
eSurf_red_type_str = ".reduction_type = integral"
eSurf_int_type_str = ".integration_type = surface"
eSurf_inter_str = ".intervals = 1"
eSurf_surf_normal_str = ".surface_normal = y"
eSurf_const_str = "my_constants."
eSurf_port_mult_vec_str = ".scaling_factor = "+str(strip_width_inv)+" "+str(strip_width_inv)+" "+str(strip_width_inv)
eSurf_red_func_name_str = ".reduced_function(x,y,z) = "

In [6]:
eSurf_port_func_str=np.empty(numYLocs_ESurfInt, dtype=object)
for l in range(0, numYLocs_ESurfInt):
    eSurf_port_func_str[l] = "\" (x > -strip_width/2. - tiny) * (x < strip_width/2. + tiny) * (y > " + eSurfName[l*numSurfsPerYLoc_ESurfInt+0] + "_Loc - dy/2. - tiny ) * (y < " + eSurfName[l*numSurfsPerYLoc_ESurfInt+0] + "_Loc + tiny) * (z > 0.) * ( z < sub_thick + tiny) *1\""
for l in range(0, 2):
    print('*******************************************')
    print('eSurf_port_func_str', eSurf_port_func_str[l])

*******************************************
eSurf_port_func_str " (x > -strip_width/2. - tiny) * (x < strip_width/2. + tiny) * (y > ESurf_L1S1_Loc - dy/2. - tiny ) * (y < ESurf_L1S1_Loc + tiny) * (z > 0.) * ( z < sub_thick + tiny) *1"
*******************************************
eSurf_port_func_str " (x > -strip_width/2. - tiny) * (x < strip_width/2. + tiny) * (y > ESurf_L2S1_Loc - dy/2. - tiny ) * (y < ESurf_L2S1_Loc + tiny) * (z > 0.) * ( z < sub_thick + tiny) *1"


# Define BField contour integral around a conductor using RawBFieldReduction

In [7]:
numSurfsPerYLoc_BSurfInt=4
numSurfs_BSurfInt = numYLocs_BSurfInt*numSurfsPerYLoc_BSurfInt
bSurfName=np.empty(numSurfs_BSurfInt, dtype=object)
reduced_diags_name_str = 'warpx.reduced_diags_names = ' 
reduced_diags_names_bfield_surface_str = ''
for l in range(0, numYLocs_BSurfInt):
    for s in range(0, numSurfsPerYLoc_BSurfInt):
        bSurfName[l*numSurfsPerYLoc_BSurfInt+s] = 'BSurf_L'+ str(l+1)+'S'+str(s+1)
        reduced_diags_names_bfield_surface_str += bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + ' '
print(reduced_diags_name_str + reduced_diags_names_bfield_surface_str)

#Choose distance
distance_uniform_BSurfInt=np.empty(numYLocs_BSurfInt, dtype=object)
distance_uniform_BSurfInt[0]='dy'
distance_uniform_BSurfInt[numYLocs_BSurfInt-1]='Ly-2*dy'
remainingLocs_BSurfInt=numYLocs_BSurfInt-2
percent_step_BSurfInt = 100./(remainingLocs_BSurfInt+1)

for l in range(1, numYLocs_BSurfInt-1):
    distance_uniform_BSurfInt[l]= str(percent_step_BSurfInt*(l)/100) + '*Ly'
    
print(distance_uniform_BSurfInt)

warpx.reduced_diags_names = BSurf_L1S1 BSurf_L1S2 BSurf_L1S3 BSurf_L1S4 BSurf_L2S1 BSurf_L2S2 BSurf_L2S3 BSurf_L2S4 BSurf_L3S1 BSurf_L3S2 BSurf_L3S3 BSurf_L3S4 BSurf_L4S1 BSurf_L4S2 BSurf_L4S3 BSurf_L4S4 BSurf_L5S1 BSurf_L5S2 BSurf_L5S3 BSurf_L5S4 BSurf_L6S1 BSurf_L6S2 BSurf_L6S3 BSurf_L6S4 BSurf_L7S1 BSurf_L7S2 BSurf_L7S3 BSurf_L7S4 BSurf_L8S1 BSurf_L8S2 BSurf_L8S3 BSurf_L8S4 BSurf_L9S1 BSurf_L9S2 BSurf_L9S3 BSurf_L9S4 BSurf_L10S1 BSurf_L10S2 BSurf_L10S3 BSurf_L10S4 
['dy' '0.1111111111111111*Ly' '0.2222222222222222*Ly'
 '0.33333333333333326*Ly' '0.4444444444444444*Ly' '0.5555555555555556*Ly'
 '0.6666666666666665*Ly' '0.7777777777777777*Ly' '0.8888888888888888*Ly'
 'Ly-2*dy']


In [8]:
# Define string constants
bSurf_type_str = ".type = RawBFieldReduction"
bSurf_red_type_str = ".reduction_type = integral"
bSurf_int_type_str = ".integration_type = surface"
bSurf_inter_str = ".intervals = 1"
bSurf_surf_normal_str = ".surface_normal = y"
bSurf_const_str = "my_constants."
bSurf_bottom_mult_vec_str = ".scaling_factor = -"+str(mu_0_inv)+" "+str(mu_0_inv)+" "+str(mu_0_inv)
bSurf_left_mult_vec_str   = ".scaling_factor =  "+str(mu_0_inv)+" "+str(mu_0_inv)+" "+str(mu_0_inv)
bSurf_top_mult_vec_str    = ".scaling_factor =  "+str(mu_0_inv)+" "+str(mu_0_inv)+" "+str(mu_0_inv)
bSurf_right_mult_vec_str  = ".scaling_factor =  "+str(mu_0_inv)+" "+str(mu_0_inv)+" -"+str(mu_0_inv)
bSurf_red_func_name_str = ".reduced_function(x,y,z) = "

bSurf_bottom_func_str=np.empty(numYLocs_BSurfInt, dtype=object)
bSurf_left_func_str=np.empty(numYLocs_BSurfInt, dtype=object)
bSurf_top_func_str=np.empty(numYLocs_BSurfInt, dtype=object)
bSurf_right_func_str=np.empty(numYLocs_BSurfInt, dtype=object)

for l in range(0, numYLocs_BSurfInt):
    bSurf_bottom_func_str[l] = "\" (x > -strip_width/2. - dz - tiny) * (x < strip_width/2. + dz + tiny) * (y > " + bSurfName[l*numSurfsPerYLoc_BSurfInt+0] + "_Loc - dy/2. - tiny ) * (y < " + bSurfName[l*numSurfsPerYLoc_BSurfInt+0] + "_Loc + tiny) * (z > sub_thick - dz - tiny) * ( z < sub_thick) *1\""
    bSurf_left_func_str[l] = "\" (x > -strip_width/2. - dz - tiny) * (x < -strip_width/2.-tiny) * (y > " + bSurfName[l*numSurfsPerYLoc_BSurfInt+1] + "_Loc - dy/2. - tiny ) * (y < " + bSurfName[l*numSurfsPerYLoc_BSurfInt+1] + "_Loc + tiny) * (z > sub_thick - dz/2.-tiny) * ( z < sub_thick + dz/2.+tiny) *1\""
    bSurf_top_func_str[l] = "\" (x > -strip_width/2. - dz - tiny) * (x < strip_width/2. + dz + tiny) * (y > " + bSurfName[l*numSurfsPerYLoc_BSurfInt+2] + "_Loc - dy/2. - tiny ) * (y < " + bSurfName[l*numSurfsPerYLoc_BSurfInt+2] + "_Loc + tiny) * (z > sub_thick) * ( z < sub_thick + dz + tiny) *1\""
    bSurf_right_func_str[l] = "\" (x > strip_width/2.+tiny) * (x < strip_width/2. + dz + tiny) * (y > " + bSurfName[l*numSurfsPerYLoc_BSurfInt+3] + "_Loc - dy/2. - tiny ) * (y < " + bSurfName[l*numSurfsPerYLoc_BSurfInt+3] + "_Loc + tiny) * (z > sub_thick - dz/2.-tiny) * ( z < sub_thick + dz/2.+tiny) *1\""

for l in range(0, 2):
    print('*******************************************')
    print('bottom surface', bSurf_bottom_func_str[l])
    print('\nright surface', bSurf_right_func_str[l])
    print('\ntop surface', bSurf_top_func_str[l])
    print('\nleft surface', bSurf_left_func_str[l])

*******************************************
bottom surface " (x > -strip_width/2. - dz - tiny) * (x < strip_width/2. + dz + tiny) * (y > BSurf_L1S1_Loc - dy/2. - tiny ) * (y < BSurf_L1S1_Loc + tiny) * (z > sub_thick - dz - tiny) * ( z < sub_thick) *1"

right surface " (x > strip_width/2.+tiny) * (x < strip_width/2. + dz + tiny) * (y > BSurf_L1S4_Loc - dy/2. - tiny ) * (y < BSurf_L1S4_Loc + tiny) * (z > sub_thick - dz/2.-tiny) * ( z < sub_thick + dz/2.+tiny) *1"

top surface " (x > -strip_width/2. - dz - tiny) * (x < strip_width/2. + dz + tiny) * (y > BSurf_L1S3_Loc - dy/2. - tiny ) * (y < BSurf_L1S3_Loc + tiny) * (z > sub_thick) * ( z < sub_thick + dz + tiny) *1"

left surface " (x > -strip_width/2. - dz - tiny) * (x < -strip_width/2.-tiny) * (y > BSurf_L1S2_Loc - dy/2. - tiny ) * (y < BSurf_L1S2_Loc + tiny) * (z > sub_thick - dz/2.-tiny) * ( z < sub_thick + dz/2.+tiny) *1"
*******************************************
bottom surface " (x > -strip_width/2. - dz - tiny) * (x < strip_width

# Define EField volume integral using FieldReduction

In [9]:
eVolumeName = np.empty(numEVolumes,dtype=object)
reduced_diags_names_efield_volume_str = ''
for v in range(0, numEVolumes):
    eVolumeName[v] = 'EVolume' + str(v+1)
    reduced_diags_names_efield_volume_str += eVolumeName[v] + ' '
print(reduced_diags_names_efield_volume_str)

efield_type_str =  ".type = FieldReduction"
efield_inter_str = ".intervals = 1"
efield_red_type_str = ".reduction_type = integral"
efield_red_func_name_str = ".reduced_function(x,y,z,Ex,Ey,Ez,Bx,By,Bz) = "
efield_full_volume_func_str = "\"if(z < sub_thick, 0.5*(Ex**2 + Ey**2 + Ez**2)*eps_0*eps_r_alumina, 0.5*(Ex**2 + Ey**2 + Ez**2)*eps_0*eps_r_air)\""

EVolume1 


# Define BField volume integral using FieldReduction

In [10]:
bVolumeName = np.empty(numBVolumes,dtype=object)
reduced_diags_names_bfield_volume_str = ''
for v in range(0, numBVolumes):
    bVolumeName[v] = 'BVolume' + str(v+1)
    reduced_diags_names_bfield_volume_str += bVolumeName[v] + ' '
print(reduced_diags_names_bfield_volume_str)

bfield_type_str =  ".type = FieldReduction"
bfield_inter_str = ".intervals = 1"
bfield_red_type_str = ".reduction_type = integral"
bfield_red_func_name_str = ".reduced_function(x,y,z,Ex,Ey,Ez,Bx,By,Bz) = "
bfield_full_volume_func_str = "\"0.5*(Bx**2+By**2+Bz**2)/mu_0\""

BVolume1 


# Final Print

In [11]:
reduced_diags_final_str = reduced_diags_name_str
if(flag_define_ESurfInt):
    reduced_diags_final_str += reduced_diags_names_efield_surface_str
if(flag_define_BSurfInt):
    reduced_diags_final_str += reduced_diags_names_bfield_surface_str
if(flag_define_EVolumeInt):
    reduced_diags_final_str += reduced_diags_names_efield_volume_str
if(flag_define_BVolumeInt):
    reduced_diags_final_str += reduced_diags_names_bfield_volume_str
print(reduced_diags_final_str)

warpx.reduced_diags_names = ESurf_L1S1 ESurf_L2S1 ESurf_L3S1 ESurf_L4S1 ESurf_L5S1 ESurf_L6S1 ESurf_L7S1 ESurf_L8S1 ESurf_L9S1 ESurf_L10S1 BSurf_L1S1 BSurf_L1S2 BSurf_L1S3 BSurf_L1S4 BSurf_L2S1 BSurf_L2S2 BSurf_L2S3 BSurf_L2S4 BSurf_L3S1 BSurf_L3S2 BSurf_L3S3 BSurf_L3S4 BSurf_L4S1 BSurf_L4S2 BSurf_L4S3 BSurf_L4S4 BSurf_L5S1 BSurf_L5S2 BSurf_L5S3 BSurf_L5S4 BSurf_L6S1 BSurf_L6S2 BSurf_L6S3 BSurf_L6S4 BSurf_L7S1 BSurf_L7S2 BSurf_L7S3 BSurf_L7S4 BSurf_L8S1 BSurf_L8S2 BSurf_L8S3 BSurf_L8S4 BSurf_L9S1 BSurf_L9S2 BSurf_L9S3 BSurf_L9S4 BSurf_L10S1 BSurf_L10S2 BSurf_L10S3 BSurf_L10S4 EVolume1 BVolume1 


In [12]:
with open(fileName, 'w') as f:
    print(reduced_diags_final_str,file=f)
    print('\n',file=f)
    if(flag_define_ESurfInt):
        print('### Defining surfaces for E-field surface integrals to obtain voltages ###',file=f)
        for l in range(0, numYLocs_ESurfInt):
            print('\n',file=f)
            for s in range(0, numSurfsPerYLoc_ESurfInt):
                print('# E-Surface ' + eSurfName[l*numSurfsPerYLoc_ESurfInt+s],file=f)
                print('\n',file=f)
                print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_type_str,file=f)
                print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_red_type_str,file=f)
                print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_int_type_str,file=f)
                print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_inter_str,file=f)
                print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_surf_normal_str,file=f)
                print(eSurf_const_str + eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + '_Loc = ' + distance_uniform_ESurfInt[l],file=f)
                if(s==0):
                    print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_port_mult_vec_str,file=f)
                    print(eSurfName[l*numSurfsPerYLoc_ESurfInt+s] + eSurf_red_func_name_str + eSurf_port_func_str[l],file=f)
        print('\n',file=f)
    if(flag_define_BSurfInt):
        print('### Defining surfaces for B-Field contour integrals to obtain current ###',file=f)
        for l in range(0, numYLocs_BSurfInt):
            print('\n',file=f)
            for s in range(0, numSurfsPerYLoc_BSurfInt):
                print('# B-Surface ' + bSurfName[l*numSurfsPerYLoc_BSurfInt+s],file=f)
                print('\n',file=f)
                print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_type_str,file=f)
                print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_red_type_str,file=f)
                print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_int_type_str,file=f)
                print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_inter_str,file=f)
                print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_surf_normal_str,file=f)
                print(bSurf_const_str + bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + '_Loc = ' + distance_uniform_BSurfInt[l],file=f)
                if(s==0):
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_bottom_mult_vec_str,file=f)
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_red_func_name_str + bSurf_bottom_func_str[l],file=f)
                elif(s==1):
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_left_mult_vec_str,file=f)
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_red_func_name_str + bSurf_left_func_str[l],file=f)
                elif(s==2):
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_top_mult_vec_str,file=f)
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_red_func_name_str + bSurf_top_func_str[l],file=f)
                elif(s==3):
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_right_mult_vec_str,file=f)
                    print(bSurfName[l*numSurfsPerYLoc_BSurfInt+s] + bSurf_red_func_name_str + bSurf_right_func_str[l],file=f)
        print('\n',file=f)
    if(flag_define_EVolumeInt):
        print('### Defining volumes for E-Field volume integrals to obtain stored magnetic energy ###',file=f)
        for v in range (0,numEVolumes):
            print('\n',file=f)
            print(eVolumeName[v] + efield_type_str,file=f)
            print(eVolumeName[v] + efield_inter_str,file=f)
            print(eVolumeName[v] + efield_red_type_str,file=f)
            print(eVolumeName[v] + efield_red_func_name_str + efield_full_volume_func_str,file=f)
        print('\n',file=f)
    if(flag_define_BVolumeInt):
        print('### Defining volumes for B-Field volume integrals to obtain stored magnetic energy ###',file=f)
        for v in range (0,numBVolumes):
            print('\n',file=f)
            print(bVolumeName[v] + bfield_type_str,file=f)
            print(bVolumeName[v] + bfield_inter_str,file=f)
            print(bVolumeName[v] + bfield_red_type_str,file=f)
            print(bVolumeName[v] + bfield_red_func_name_str + bfield_full_volume_func_str,file=f)